In [ ]:
# Durga Yasasvi Y,   IMT2016060.
# Srujan Swaroop G,  IMT2016033.
# Siddharth Reddy D, IMT2016037.

from keras.models import Model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
from PIL import Image
import numpy as np
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets import mnist
from keras.callbacks import TensorBoard
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from sklearn.metrics.pairwise import euclidean_distances
from matplotlib import pyplot as plt
from numpy import linalg as LA
import random
import numpy as np
import pandas as pd
import os
import cv2
from keras.models import Model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dropout, Activation
from PIL import Image
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


Using TensorFlow backend.


In [89]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def train_split(l,train_percent):
    random.seed(2019)
    random.shuffle(l)
    cut = int(len(l)*train_percent)
    return l[:cut],l[cut:]

In [0]:

data_path ="../content/drive/My Drive/AVR_data/"
folder_path = os.listdir(data_path)
folder_path = [data_path+x for x in folder_path]
train_imgs = []
test_imgs = []
count = 0
train_labels = np.array([])
test_labels = np.array([])
a=0
for path in folder_path:
    image_names = os.listdir(path)
    image_names = [path +"/"+x for x in image_names ]
    a+=len(image_names)
    tra,tes = train_split(image_names,1)
    train_imgs=train_imgs + tra
    test_imgs=test_imgs +tes
    for i in tra:
        train_labels = np.append(train_labels, count)
    for i in tes:
        test_labels = np.append(test_labels, count)
    count= count+1

In [0]:


train_images = []
for i in train_imgs:
    img = cv2.imread(i)
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    train_images.append(img)

test_images = []    
for i in test_imgs:
    img = cv2.imread(i)
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    test_images.append(img)
  

In [0]:

train_images = [np.rollaxis(x, 2, 0) for x in train_images]
test_images = [np.rollaxis(x, 2, 0) for x in test_images]

In [0]:
all_image = train_images
all_image_labels = train_labels

In [0]:
def vgg_face(weights_path=None):
    keras.backend.set_image_data_format('channels_first')
    img = Input(shape=(3, 224, 224))

    pad1_1 = ZeroPadding2D(padding=(1, 1))(img)
    conv1_1 = Convolution2D(64, 3, 3, activation='relu', name='conv1_1')(pad1_1)
    pad1_2 = ZeroPadding2D(padding=(1, 1))(conv1_1)
    conv1_2 = Convolution2D(64, 3, 3, activation='relu', name='conv1_2')(pad1_2)
    pool1 = MaxPooling2D((2, 2), strides=(2, 2),dim_ordering="th")(conv1_2)

    pad2_1 = ZeroPadding2D((1, 1))(pool1)
    conv2_1 = Convolution2D(128, 3, 3, activation='relu', name='conv2_1')(pad2_1)
    pad2_2 = ZeroPadding2D((1, 1))(conv2_1)
    conv2_2 = Convolution2D(128, 3, 3, activation='relu', name='conv2_2')(pad2_2)
    pool2 = MaxPooling2D((2, 2), strides=(2, 2),dim_ordering="th")(conv2_2,)

    pad3_1 = ZeroPadding2D((1, 1))(pool2)
    conv3_1 = Convolution2D(256, 3, 3, activation='relu', name='conv3_1')(pad3_1)
    pad3_2 = ZeroPadding2D((1, 1))(conv3_1)
    conv3_2 = Convolution2D(256, 3, 3, activation='relu', name='conv3_2')(pad3_2)
    pad3_3 = ZeroPadding2D((1, 1))(conv3_2)
    conv3_3 = Convolution2D(256, 3, 3, activation='relu', name='conv3_3')(pad3_3)
    pool3 = MaxPooling2D((2, 2), strides=(2, 2),dim_ordering="th")(conv3_3)

    pad4_1 = ZeroPadding2D((1, 1))(pool3)
    conv4_1 = Convolution2D(512, 3, 3, activation='relu', name='conv4_1')(pad4_1)
    pad4_2 = ZeroPadding2D((1, 1))(conv4_1)
    conv4_2 = Convolution2D(512, 3, 3, activation='relu', name='conv4_2')(pad4_2)
    pad4_3 = ZeroPadding2D((1, 1))(conv4_2)
    conv4_3 = Convolution2D(512, 3, 3, activation='relu', name='conv4_3')(pad4_3)
    pool4 = MaxPooling2D((2, 2), strides=(2, 2),dim_ordering="th")(conv4_3)

    pad5_1 = ZeroPadding2D((1, 1))(pool4)
    conv5_1 = Convolution2D(512, 3, 3, activation='relu', name='conv5_1')(pad5_1)
    pad5_2 = ZeroPadding2D((1, 1))(conv5_1)
    conv5_2 = Convolution2D(512, 3, 3, activation='relu', name='conv5_2')(pad5_2)
    pad5_3 = ZeroPadding2D((1, 1))(conv5_2)
    conv5_3 = Convolution2D(512, 3, 3, activation='relu', name='conv5_3')(pad5_3)
    pool5 = MaxPooling2D((2, 2), strides=(2, 2),dim_ordering="th")(conv5_3)

    fc6 = Convolution2D(4096, 7, strides= 7, activation='relu', name='fc6',data_format='channels_first')(pool5)
    fc6_drop = Dropout(0.5)(fc6)
    fc7 = Convolution2D(4096, 1, 1, activation='relu', name='fc7')(fc6_drop)
    fc7_drop = Dropout(0.5)(fc7)
    fc8 = Convolution2D(2622, 1, 1, name='fc8')(fc7_drop)
    flat = Flatten()(fc8)
    out = Activation('softmax')(flat)

    model = Model(input=img, output=out)

    if weights_path:
        model.load_weights(weights_path)

    return model


In [0]:
model = vgg_face('../content/drive/My Drive/vgg-face-keras.h5')


In [0]:
def getVec(Img):
    Img = np.array(Img).astype(np.float32)
    Img = np.expand_dims(Img, axis=0)
    out = model.predict(Img)
    return out

In [0]:
def Dist(vec1,vec2):
  return np.linalg.norm(vec1-vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))

In [0]:
data_path ="../content/drive/My Drive/AVR_data/"
folder_path = os.listdir(data_path)
folder_path = [data_path+x for x in folder_path]
train_imgs = []
test_imgs = []
count = 0
train_labels = np.array([])
test_labels = np.array([])
a=0
dis = []
for path in folder_path:
    image_names = os.listdir(path)
    image_names = [path +"/"+x for x in image_names ]
#     print(image_names)
    for i in range(0,len(image_names)):
      for j in range(i,len(image_names)):

        img1 = cv2.imread(image_names[i])
        img2 = cv2.imread(image_names[j])
      
        img1 = cv2.resize(img1,(224,224),interpolation = cv2.INTER_CUBIC)
        img2 = cv2.resize(img2,(224,224),interpolation = cv2.INTER_CUBIC)

        img1 = np.rollaxis(img1, 2, 0)
        img2 = np.rollaxis(img2, 2, 0)
      
        dis.append(Dist(img1,img2))


In [0]:
data_path ="./../content/drive/My Drive/AVR_data/"
folder_path = os.listdir(data_path)
folder_path = [data_path+x for x in folder_path]
train_imgs = []
test_imgs = []
count = 0
train_labels = np.array([])
test_labels = np.array([])
a=0
random_dis = []

for i in range(0,9500):
  path1 = random.choice(folder_path)
  path2 = random.choice(folder_path)
  
  
  image_names1 = os.listdir(path1)
  image_names1 = [path1 +"/"+x for x in image_names1 ]

  image_names2 = os.listdir(path2)
  image_names2 = [path2 +"/"+x for x in image_names2 ]

  final_image1 = random.choice(image_names1)
  final_image2 = random.choice(image_names2)
  
  img1 = cv2.imread(final_image1)
  img2 = cv2.imread(final_image2)

      
  img1 = cv2.resize(img1,(224,224),interpolation = cv2.INTER_CUBIC)
  img2 = cv2.resize(img2,(224,224),interpolation = cv2.INTER_CUBIC)

  img1 = np.rollaxis(img1, 2, 0)
  img2 = np.rollaxis(img2, 2, 0)
      
  if(path1==path2):
    i = i-1  
  else:
    random_dis.append(Dist(img1,img2))
    
    

In [0]:
X = []
for i in dis:
  X.append(i)
for i in random_dis:
  X.append(i)
y = []
for i in dis:
  y.append(1)
for i in random_dis:
  y.append(0)
  

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
X_train = np.array(X_train).reshape((len(X_train),1))
X_test = np.array(X_test).reshape((len(X_test),1))

In [0]:
from sklearn.svm import SVC # "Support Vector Classifier" 
clf = SVC(kernel='linear') 

In [106]:
clf.fit(X_train, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [0]:
 accuracy = clf.score(X_test, y_test)

In [122]:
print('Acuuracy Score: ', accuracy * 100, '%')

Acuuracy Score:  82.16870342771982 %
